In [ ]:
# Imports
import numpy as np
import cv2
import glob
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import matplotlib.ticker as plticker

In [ ]:
images = glob.glob('../CarND-Advanced-Lane-Lines/test_images/*.jpg')

In [ ]:
########################################################
# 1. Compute the camera calibration matrix 
#    and distortion coefficients given a set of chessboard images.
########################################################

# prepare object points
nx = 9
ny = 6

# prepare object points, like (0,0,0), (1,0,0), (2,0,0) ....,(6,5,0)
objp = np.zeros((nx*ny,3), np.float32)
objp[:,:2] = np.mgrid[0:nx, 0:ny].T.reshape(-1,2)

# Arrays to store object points and image points from all the images.
objpoints = [] # 3d points in real world space
imgpoints = [] # 2d points in image plane.

# Make a list of calibration images
calibration_images = glob.glob('../CarND-Advanced-Lane-Lines/camera_cal/calibration*.jpg')

# Step through the list and search for chessboard corners
for idx, fname in enumerate(calibration_images):
    img = cv2.imread(fname)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # Find the chessboard corners
    ret, corners = cv2.findChessboardCorners(gray, (nx,ny), None)
    #print(idx, ret)
    
    # If found, add object points, image points
    if ret == True:
        objpoints.append(objp)
        imgpoints.append(corners)


image = cv2.imread('../CarND-Advanced-Lane-Lines/camera_cal/calibration3.jpg')
image_size = (image.shape[1], image.shape[0])

# Do camera calibration given object points and image points
ret, mtx, dist_coeffs, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, image_size,None,None)

print("Calibrated Camera.")

In [ ]:
########################################################
# 2. Apply a distortion correction to raw images.
########################################################

# TEST on Camera Calibration Image
image = cv2.imread('../CarND-Advanced-Lane-Lines/camera_cal/calibration3.jpg')

def undistort(img):
    dst = cv2.undistort(img, mtx, dist_coeffs, None, mtx)
    return dst

# Camera Calibration image Undistortion
undist = undistort(image)

f, (ax1, ax2) = plt.subplots(1, 2, figsize=(25, 10))
f.tight_layout()
ax1.imshow(image)
ax1.set_title('Original Image', fontsize=50)
ax2.imshow(undist)
ax2.set_title('Undistorted Image', fontsize=50)
plt.subplots_adjust(left=0., right=1, top=0.9, bottom=0.)


# TEST
# Given driving images Undistortion
for idx, fname in enumerate(images):    
    img = cv2.imread(fname)

    undst = undistort(img)

    f, (ax1, ax2) = plt.subplots(1, 2, figsize=(25, 10))
    f.tight_layout()
    
    ax1.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
    ax1.set_title('Original Image', fontsize=50)
    
    ax2.imshow(cv2.cvtColor(undst, cv2.COLOR_BGR2RGB))
    ax2.set_title('Undistorted Image', fontsize=50)
    
    plt.subplots_adjust(left=0., right=1, top=0.9, bottom=0.)
    

In [ ]:
########################################################
# 3. Use color transforms, gradients, etc., 
#    to create a thresholded binary image.
########################################################

# Function that takes an image, gradient orientation,
# and threshold min / max values.
def abs_sobel_thresh(img, orient='x', thresh=(0, 255)):
    thresh_min = thresh[0]
    thresh_max = thresh[1]
    # Convert to grayscale
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    # Apply x or y gradient with the OpenCV Sobel() function
    # and take the absolute value
    if orient == 'x':
        abs_sobel = np.absolute(cv2.Sobel(gray, cv2.CV_64F, 1, 0))
    if orient == 'y':
        abs_sobel = np.absolute(cv2.Sobel(gray, cv2.CV_64F, 0, 1))
    # Rescale back to 8 bit integer
    scaled_sobel = np.uint8(255*abs_sobel/np.max(abs_sobel))
    # Create a copy and apply the threshold
    binary_output = np.zeros_like(scaled_sobel)

    binary_output[(scaled_sobel >= thresh_min) & (scaled_sobel <= thresh_max)] = 1

    # Return the result
    return binary_output

# Function to return the magnitude of the gradient
# for a given sobel kernel size and threshold values
def mag_thresh(img, sobel_kernel=3, mag_thresh=(0, 255)):
    # Convert to grayscale
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    # Take both Sobel x and y gradients
    sobelx = cv2.Sobel(gray, cv2.CV_64F, 1, 0, ksize=sobel_kernel)
    sobely = cv2.Sobel(gray, cv2.CV_64F, 0, 1, ksize=sobel_kernel)
    # Calculate the gradient magnitude
    gradmag = np.sqrt(sobelx**2 + sobely**2)
    # Rescale to 8 bit
    scale_factor = np.max(gradmag)/255
    gradmag = (gradmag/scale_factor).astype(np.uint8)
    # Create a binary image of ones where threshold is met, zeros otherwise
    binary_output = np.zeros_like(gradmag)
    binary_output[(gradmag >= mag_thresh[0]) & (gradmag <= mag_thresh[1])] = 1

    # Return the binary image
    return binary_output

# Function to threshold an image for a given range and Sobel kernel
def dir_threshold(img, sobel_kernel=3, thresh=(0, np.pi/2)):
    # Grayscale
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    # Calculate the x and y gradients
    sobelx = cv2.Sobel(gray, cv2.CV_64F, 1, 0, ksize=sobel_kernel)
    sobely = cv2.Sobel(gray, cv2.CV_64F, 0, 1, ksize=sobel_kernel)
    # Take the absolute value of the gradient direction,
    # apply a threshold, and create a binary image result
    absgraddir = np.arctan2(np.absolute(sobely), np.absolute(sobelx))
    binary_output =  np.zeros_like(absgraddir)
    binary_output[(absgraddir >= thresh[0]) & (absgraddir <= thresh[1])] = 1

    # Return the binary image
    return binary_output

def s_thresh(img, thresh=(0, 255)):
    # Convert to HLS color space and separate the S channel
    hls = cv2.cvtColor(img, cv2.COLOR_RGB2HLS)
    s_channel = hls[:,:,2]

    s_thresh_min = thresh[0]
    s_thresh_max = thresh[1]
    
    # Threshold color channel
    s_binary = np.zeros_like(s_channel)
    s_binary[(s_channel >= s_thresh_min) & (s_channel <= s_thresh_max)] = 1
    
    return s_binary

# COMBINATION #1
# Combine all Gradient Thresholds
def grad_thresholds_only(img):
    # Sobel kernel size
    ksize = 15 

    # Apply each of the thresholding functions
    gradx = abs_sobel_thresh(img, orient='x', thresh=(20, 100))
    grady = abs_sobel_thresh(img, orient='y', thresh=(20, 100))
    mag_binary = mag_thresh(img, sobel_kernel=ksize, mag_thresh=(30, 100))
    dir_binary = dir_threshold(img, sobel_kernel=ksize, thresh=(0.7, 1.3))

    combined_grad = np.zeros_like(gradx)
    combined_grad[((gradx == 1) & (grady == 1)) | ((mag_binary == 1) & (dir_binary == 1))] = 1

    return combined_grad

# COMBINATION #2
# Combine the two binary thresholds - Gradient in X | S-channel thresholding
def grads_with_s_channel(img):
    gradx = abs_sobel_thresh(img, orient='x', thresh=(20, 100))
    
    s_thresh_min = 170
    s_thresh_max = 255
    s_binary = s_thresh(img, (s_thresh_min, s_thresh_max))
    
    combined_x_s = np.zeros_like(gradx)
    combined_x_s[(s_binary == 1) | (gradx == 1)] = 1
    
    return combined_x_s

# COMBINATION #3
# Combine: Multiple Gradient Thresholds | S-channel thresholding
def all_grads_with_s_channel(img):
    combined_grad = grad_thresholds_only(img)
    
    s_thresh_min = 170
    s_thresh_max = 255
    s_binary = s_thresh(img, (s_thresh_min, s_thresh_max))
    
    combined_grad_s = np.zeros_like(combined_grad)
    combined_grad_s[(combined_grad == 1) | (s_binary == 1)] = 1

    return combined_grad_s
    

# TEST:
for idx, fname in enumerate(images):
    
    img = cv2.imread(fname)
    
    combined_grad = grad_thresholds_only(img)
    
    combined_x_s = grads_with_s_channel(img)

    combined_grad_s = all_grads_with_s_channel(img)
        
    # Plot thresholded images
    f, ([ax1, ax2], [ax3, ax4]) = plt.subplots(2, 2, figsize=(20,10))
    
    ax1.set_title('Original image')
    ax1.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))

    ax2.set_title('Combined Gradients')
    ax2.imshow(combined_grad, cmap='gray')
    
    ax3.set_title('Combined Gradient-X and S-channel')
    ax3.imshow(combined_x_s, cmap='gray')
    
    ax4.set_title('Combined Gradients and and S-channel')
    ax4.imshow(combined_grad_s, cmap='gray')


In [ ]:
########################################################
# 4. Apply a perspective transform 
#    to rectify binary image ("birds-eye view").
########################################################

import numpy as np
import cv2
import matplotlib.pyplot as plt

def binary_transform_top_down(img):
        
    # First, get the thresholded binary image
    binary = grads_with_s_channel(img)

    width = img.shape[1]
    height = img.shape[0]
    
    # Now, Perspective-Transform 
    topleft = (595,450)
    topright = (687,450)
    bottomleft = (200,720)
    bottomright = (1100,720)
    src = np.float32([topleft,topright,bottomleft,bottomright])
    
    # Destination points
    dst = np.float32([[200,0],[1100,0],[200,720],[1100,720]])

    M = cv2.getPerspectiveTransform(src, dst)

    # get top-down view
    top_down = cv2.warpPerspective(binary, M, (width,height), flags=cv2.INTER_LINEAR)

    return top_down

# TEST:
for idx, fname in enumerate(images):
    
    img = undistort(cv2.imread(fname))
    
    binary_warped_top_down = binary_transform_top_down(img)
    
    f, (ax1, ax2) = plt.subplots(1, 2, figsize=(24, 9))
    f.tight_layout()

    ax1.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
    ax1.set_title('Original Image', fontsize=50)

    ax2.imshow(binary_warped_top_down, cmap='gray')
    ax2.set_title('Top-Down Warped Image', fontsize=50)
    
    plt.subplots_adjust(left=0., right=1, top=0.9, bottom=0.)


In [ ]:
########################################################
# 5. Detect lane pixels and fit to find the lane boundary.
# 6. Determine the curvature of the lane and vehicle position 
#    with respect to center.
########################################################

def get_lane_curvature(img):
    
    binary_warped = binary_transform_top_down(img)

    # Assuming you have created a warped binary image called "binary_warped"
    # Take a histogram of the bottom half of the image
    histogram = np.sum(binary_warped[binary_warped.shape[0]//2:,:], axis=0)
    
    # Create an output image to draw on and  visualize the result
    out_img = np.dstack((binary_warped, binary_warped, binary_warped))*255
    
    # Find the peak of the left and right halves of the histogram
    # These will be the starting point for the left and right lines
    midpoint = np.int(histogram.shape[0]//2)
    leftx_base = np.argmax(histogram[:midpoint])
    rightx_base = np.argmax(histogram[midpoint:]) + midpoint

    # Choose the number of sliding windows
    nwindows = 9
    # Set height of windows
    window_height = np.int(binary_warped.shape[0]//nwindows)
    # Identify the x and y positions of all nonzero pixels in the image
    nonzero = binary_warped.nonzero()
    nonzeroy = np.array(nonzero[0])
    nonzerox = np.array(nonzero[1])
    # Current positions to be updated for each window
    leftx_current = leftx_base
    rightx_current = rightx_base
    # Set the width of the windows +/- margin
    margin = 100
    # Set minimum number of pixels found to recenter window
    minpix = 50
    # Create empty lists to receive left and right lane pixel indices
    left_lane_inds = []
    right_lane_inds = []

    # Step through the windows one by one
    for window in range(nwindows):
        # Identify window boundaries in x and y (and right and left)
        win_y_low = binary_warped.shape[0] - (window+1)*window_height
        win_y_high = binary_warped.shape[0] - window*window_height
        win_xleft_low = leftx_current - margin
        win_xleft_high = leftx_current + margin
        win_xright_low = rightx_current - margin
        win_xright_high = rightx_current + margin
        # Draw the windows on the visualization image
        cv2.rectangle(out_img,(win_xleft_low,win_y_low),(win_xleft_high,win_y_high),
        (0,255,0), 2) 
        cv2.rectangle(out_img,(win_xright_low,win_y_low),(win_xright_high,win_y_high),
        (0,255,0), 2) 
        # Identify the nonzero pixels in x and y within the window
        good_left_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & 
        (nonzerox >= win_xleft_low) &  (nonzerox < win_xleft_high)).nonzero()[0]
        good_right_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & 
        (nonzerox >= win_xright_low) &  (nonzerox < win_xright_high)).nonzero()[0]
        # Append these indices to the lists
        left_lane_inds.append(good_left_inds)
        right_lane_inds.append(good_right_inds)
        # If you found > minpix pixels, recenter next window on their mean position
        if len(good_left_inds) > minpix:
            leftx_current = np.int(np.mean(nonzerox[good_left_inds]))
        if len(good_right_inds) > minpix:        
            rightx_current = np.int(np.mean(nonzerox[good_right_inds]))

    # Concatenate the arrays of indices
    left_lane_inds = np.concatenate(left_lane_inds)
    right_lane_inds = np.concatenate(right_lane_inds)

    # Extract left and right line pixel positions
    leftx = nonzerox[left_lane_inds]
    lefty = nonzeroy[left_lane_inds] 
    rightx = nonzerox[right_lane_inds]
    righty = nonzeroy[right_lane_inds] 
    
    # Fit a second order polynomial to each
    left_fit = np.polyfit(lefty, leftx, 2)
    right_fit = np.polyfit(righty, rightx, 2)

    # Generate x and y values for plotting
    ploty = np.linspace(0, binary_warped.shape[0]-1, binary_warped.shape[0] )
    left_fitx = left_fit[0]*ploty**2 + left_fit[1]*ploty + left_fit[2]
    right_fitx = right_fit[0]*ploty**2 + right_fit[1]*ploty + right_fit[2]

    # OUT IMAGE # 1
    
    out_img[nonzeroy[left_lane_inds], nonzerox[left_lane_inds]] = [255, 0, 0]
    out_img[nonzeroy[right_lane_inds], nonzerox[right_lane_inds]] = [0, 0, 255]

#     plt.imshow(out_img)
#     plt.plot(left_fitx, ploty, color='yellow')
#     plt.plot(right_fitx, ploty, color='yellow')
#     plt.xlim(0, 1280)
#     plt.ylim(720, 0)
    
    # OUT IMAGE # 2
    
    # Create an image to draw on and an image to show the selection window
    out_img2 = np.dstack((binary_warped, binary_warped, binary_warped))*255
    window_img = np.zeros_like(out_img2)
    # Color in left and right line pixels
    out_img2[nonzeroy[left_lane_inds], nonzerox[left_lane_inds]] = [255, 0, 0]
    out_img2[nonzeroy[right_lane_inds], nonzerox[right_lane_inds]] = [0, 0, 255]

    # Generate a polygon to illustrate the search window area
    # And recast the x and y points into usable format for cv2.fillPoly()
    left_line_window1 = np.array([np.transpose(np.vstack([left_fitx-margin, ploty]))])
    left_line_window2 = np.array([np.flipud(np.transpose(np.vstack([left_fitx+margin, 
                                  ploty])))])
    left_line_pts = np.hstack((left_line_window1, left_line_window2))
    right_line_window1 = np.array([np.transpose(np.vstack([right_fitx-margin, ploty]))])
    right_line_window2 = np.array([np.flipud(np.transpose(np.vstack([right_fitx+margin, 
                                  ploty])))])
    right_line_pts = np.hstack((right_line_window1, right_line_window2))

    # Draw the lane onto the warped blank image
    cv2.fillPoly(window_img, np.int_([left_line_pts]), (0,255, 0))
    cv2.fillPoly(window_img, np.int_([right_line_pts]), (0,255, 0))
    result = cv2.addWeighted(out_img2, 1, window_img, 0.3, 0)
    
#     plt.imshow(result)
#     plt.plot(left_fitx, ploty, color='yellow')
#     plt.plot(right_fitx, ploty, color='yellow')
#     plt.xlim(0, 1280)
#     plt.ylim(720, 0)
    
    return (ploty, left_fit, left_fitx, right_fit, right_fitx, out_img, result, leftx, rightx)

In [ ]:
########################################################
# TEST 5. & 6. Check lane curvature
########################################################

#TEST:
for idx, fname in enumerate(images):
    img = undistort(cv2.imread(fname))
    
    binary_warped_top_down = binary_transform_top_down(img)
    ploty, left_fit, left_fitx, right_fit, right_fitx, out_img, out_img2, leftx, rightx = get_lane_curvature(img)
    
    f, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(24, 9))
    f.tight_layout()

    ax1.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
    ax1.set_title('Original Image', fontsize=12)
    
    ax2.imshow(binary_warped_top_down, cmap='gray')
    ax2.set_title('Top-Down Warped Image', fontsize=12)

    ax3.imshow(out_img)
    ax3.plot(left_fitx, ploty, color='yellow')
    ax3.plot(right_fitx, ploty, color='yellow')
    ax3.set_xlim([0,1280])
    ax3.set_ylim([720,0])
    ax3.set_title('Lanes wth identified curvature', fontsize=12)

    ax4.imshow(out_img2)
    ax4.plot(left_fitx, ploty, color='yellow')
    ax4.plot(right_fitx, ploty, color='yellow')
    ax4.set_xlim([0,1280])
    ax4.set_ylim([720,0])
    ax4.set_title('Lanes wth identified curvature', fontsize=12)
    
    plt.subplots_adjust(left=0., right=1, top=0.9, bottom=0.)


In [ ]:
########################################################
# 7. Warp the detected lane boundaries back onto the original image.
# 8. Output visual display of the lane boundaries 
########################################################

from numpy.linalg import inv

def unwarp_lane_boundaries(img, binary_lane_image):
    #print(img.shape) # (720, 1280, 3)
    width = img.shape[1]
    height = img.shape[0]
    
    # Now, Perspective-Transform 
    topleft = (200,0)
    topright = (1100,0)
    bottomleft = (200,720)
    bottomright = (1100,720)
    src = np.float32([topleft,topright,bottomleft,bottomright])
    
    # Destination points
    dst = np.float32([[200,0],[1100,0],[200,720],[1100,720]])
    dst = np.float32([[595,450],[690,450],[200,720],[1100,720]])

    M = cv2.getPerspectiveTransform(src, dst)

    # get UN-WARPED view
    unwarped = cv2.warpPerspective(binary_lane_image, M, (width,height), flags=cv2.INTER_LINEAR)
        
    return (unwarped, M)

def mask(unwarped_lane_boundaries):
    mask = np.zeros(unwarped_lane_boundaries.shape, dtype=np.uint8)
    roi_corners = np.array([[(500,450),(800,450),(1200,720),(100,720)]], dtype=np.int32)
    channel_count = unwarped_lane_boundaries.shape[2]
    ignore_mask_color = (255,)*channel_count
    cv2.fillPoly(mask, roi_corners, ignore_mask_color)
    masked = cv2.bitwise_and(unwarped_lane_boundaries, mask)

    # Now make it white - except inside roi corners
    white_out = np.zeros(unwarped_lane_boundaries.shape, dtype=np.uint8)
    out_corners = np.array([[(1280,0),(0,0),(0,720),(100,720),(500,450),(800,450),(1200,720),(1280,720)]], dtype=np.int32)
    cv2.fillPoly(masked, out_corners, ignore_mask_color)
    
    return masked

def project_detected_lanes(undist, warped, left_fitx, right_fitx, ploty, M):
    # Create an image to draw the lines on
    warp_zero = np.zeros_like(warped).astype(np.uint8)
    color_warp = np.dstack((warp_zero, warp_zero, warp_zero))
    
    #print(warped.shape)
    #print(color_warp.shape)
    
    # Recast the x and y points into usable format for cv2.fillPoly()
    pts_left = np.array([np.transpose(np.vstack([left_fitx, ploty]))])
    pts_right = np.array([np.flipud(np.transpose(np.vstack([right_fitx, ploty])))])
    pts = np.hstack((pts_left, pts_right))
    
    # Draw the lane onto the warped blank image
    cv2.fillPoly(color_warp, np.int_([pts]), (0,255, 0))
    
    # Warp the blank back to original image space using inverse perspective matrix (Minv)
    #newwarp = cv2.warpPerspective(color_warp, Minv, (image.shape[1], image.shape[0])) 
    #newwarp = cv2.warpPerspective(color_warp, inv(mtx), (undist.shape[1], undist.shape[0])) 
    newwarp = cv2.warpPerspective(color_warp, M, (undist.shape[1], undist.shape[0])) 
    
    # Combine the result with the original image
    result = cv2.addWeighted(undist, 1, newwarp, 0.3, 0)
    
    return (newwarp, result)

def unwarp_and_overlay(img):
    
    ploty, left_fit, left_fitx, right_fit, right_fitx, out_img, out_img2, leftx, rightx = get_lane_curvature(img)
    
    unwarped_lane_boundaries, M = unwarp_lane_boundaries(img, out_img2)
    
    # Not needed any more
    #masked = mask(unwarped_lane_boundaries)
    
    warped = binary_transform_top_down(img)
    
    projection, overlayed = project_detected_lanes(img, warped, left_fitx, right_fitx, ploty, M)
    
    return (unwarped_lane_boundaries, projection, overlayed)


# TEST
for idx, fname in enumerate(images):
    img = undistort(cv2.imread(fname))
    
    unwarped_lane_boundaries, projection, overlayed = unwarp_and_overlay(img)
    
    f, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(24, 9))
    f.tight_layout()

    ax1.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
    ax1.set_title('Original Image', fontsize=20)
    
    ax2.imshow(unwarped_lane_boundaries)
    ax2.set_title('Unwarped Lane Boundaries', fontsize=20)

    ax3.imshow(projection)
    ax3.set_title('Projection', fontsize=20)
    
    ax4.imshow(cv2.cvtColor(overlayed, cv2.COLOR_BGR2RGB))
    ax4.set_title('Overlayed with lanes', fontsize=20)
    
    plt.subplots_adjust(left=0., right=1, top=0.9, bottom=0.)


In [ ]:
########################################################
# 9. Output numerical estimation of lane curvature 
#    and vehicle position.
########################################################

def get_curve_m_and_position(img):
    # Pixel-space
    ploty, left_fit, left_fitx, right_fit, right_fitx, out_img, out_img2, leftx, rightx = get_lane_curvature(img)
    
    y_eval = np.max(ploty)
    left_curverad = ((1 + (2*left_fit[0]*y_eval + left_fit[1])**2)**1.5) / np.absolute(2*left_fit[0])
    right_curverad = ((1 + (2*right_fit[0]*y_eval + right_fit[1])**2)**1.5) / np.absolute(2*right_fit[0])
    #print(left_curverad, right_curverad)

    # Meters-space
    # Define conversions in x and y from pixels space to meters
    ym_per_pix = 30/720 # meters per pixel in y dimension
#    xm_per_pix = 3.7/700 # meters per pixel in x dimension
    xm_per_pix = 3.7/900 # meters per pixel in x dimension

    # Fit new polynomials to x,y in world space
    left_fit_cr = np.polyfit(ploty*ym_per_pix, left_fitx*xm_per_pix, 2)
    right_fit_cr = np.polyfit(ploty*ym_per_pix, right_fitx*xm_per_pix, 2)

    # Calculate the new radii of curvature
    left_curverad_m = ((1 + (2*left_fit_cr[0]*y_eval*ym_per_pix + left_fit_cr[1])**2)**1.5) / np.absolute(2*left_fit_cr[0])
    right_curverad_m = ((1 + (2*right_fit_cr[0]*y_eval*ym_per_pix + right_fit_cr[1])**2)**1.5) / np.absolute(2*right_fit_cr[0])    
    
    # Get position off-center
    left_lane_bottom = left_fit[0]*y_eval**2 + left_fit[1]*y_eval + left_fit[2]
    right_lane_bottom = right_fit[0]*y_eval**2 + right_fit[1]*y_eval + right_fit[2]

    vehicle_centre = left_lane_bottom + ((right_lane_bottom - left_lane_bottom)/2)
    offset = vehicle_centre - (1280/2)
    offset_m = offset*xm_per_pix
    
    #print(left_curverad_m, 'm', right_curverad_m, 'm', offset_m, 'm')
    
    return (left_curverad_m, right_curverad_m, offset_m)

def get_and_overlay_stats(img):

    left_curverad_m, right_curverad_m, offset_m = get_curve_m_and_position(img)

    unwarped_lane_boundaries, masked, overlayed = unwarp_and_overlay(img)
    
    avg_curvature = (left_curverad_m + right_curverad_m)/2
    curvature_txt = 'Road curvature radius: ' + str(round(avg_curvature)) + 'm'
    
    if offset_m < 0:
        position_txt = 'Vehicle is ' + str(round(offset_m, 2)) + 'm right of center'
    else:
        position_txt = 'Vehicle is ' + str(round(offset_m,2)) + 'm left of center'
    
    font = cv2.FONT_HERSHEY_SIMPLEX
    cv2.putText(overlayed, curvature_txt,(10,100), font, 2,(100,38,216),2,cv2.LINE_AA)
    cv2.putText(overlayed, position_txt,(10,200), font, 1.5,(100,38,216),2,cv2.LINE_AA)
    
    return overlayed


# TEST
for idx, fname in enumerate(images):
    img = undistort(cv2.imread(fname))
    
    overlayed = get_and_overlay_stats(img)
    
    f, ax1 = plt.subplots(1, 1, figsize=(24, 9))
    f.tight_layout()

    ax1.imshow(cv2.cvtColor(overlayed, cv2.COLOR_BGR2RGB))
    
    plt.subplots_adjust(left=0., right=1, top=0.9, bottom=0.)

    

In [ ]:
########################################################
# 9. Generate video with lane-overlays, lane-curvature, 
#    and vehicle position
########################################################

from moviepy.editor import VideoFileClip
from IPython.display import HTML

def process_image(image):
    img = undistort(image)
    overlayed = get_and_overlay_stats(img)
    return overlayed

output = 'project_video_output.mp4'
clip1 = VideoFileClip("../CarND-Advanced-Lane-Lines/project_video.mp4")
clip = clip1.fl_image(process_image)
%time clip.write_videofile(output, audio=False)